In [1]:
import pandas as pd
from html import unescape
import re
import numpy as np

from nltk import tokenize as tknz
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

import pickle

In [2]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

In [3]:
data

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love yoyou take with yoyou all the time ...,"[model, love, yoyou, take, with, yoyou, all, t...","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"
...,...,...,...,...,...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory leftright polarisation trump u...,"[thought, factory, leftright, polarisation, tr...","[thought, factory, leftright, polarisation, tr...","[thought, factori, leftright, polaris, trump, ...","[think, factory, leftright, polarisation, trum..."
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,...","[feel, like, mermaid, hairflip, neverreadi, fo...","[feel, like, mermaid, hairflip, neverready, fo..."
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohioomg amp used w...,"[hillary, campaigned, today, in, ohioomg, amp,...","[hillary, campaigned, today, ohioomg, amp, use...","[hillari, campaign, today, ohioomg, amp, use, ...","[hillary, campaign, today, ohioomg, amp, use, ..."
49157,49158,NaN,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","[happi, work, confer, right, mindset, lead, cu...","[happy, work, conference, right, mindset, lead..."


**1. Создайте мешок слов с помощью `sklearn.feature_extraction.text.CountVectorizer.fit_transform()`. Применим его к `'tweet_stemmed'` и `'tweet_lemmatized'` отдельно.**

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью `max_df`.

●	Ограничим количество слов, попадающий в мешок, с помощью `max_features` = 1000.

●	Исключим стоп-слова с помощью `stop_words='english'`.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью `CountVectorizer.get_feature_names()`.


In [4]:
corpus_stemmed = [','.join(i) for i in data['tweet_stemmed']]
count_vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_stemmed = count_vectorizer.fit_transform(corpus_stemmed)

In [5]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
corpus_lemmatized = [','.join(i) for i in data['tweet_lemmatized']]
count_vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_lemmatized = count_vectorizer.fit_transform(corpus_lemmatized)

In [7]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_lemmatized.toarray(), columns = feature_names)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yrs,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**2. Создайте мешок слов с помощью `sklearn.feature_extraction.text.TfidfVectorizer.fit_transform()`. Применим его к `'tweet_stemmed'` и `'tweet_lemmatized'` отдельно.**

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью `max_df`.

●	Ограничим количество слов, попадающий в мешок, с помощью `max_features` = 1000.

●	Исключим стоп-слова с помощью `stop_words='english'`.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [8]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_stemmed = tfidf_vectorizer.fit_transform(corpus_stemmed)

In [9]:
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.743324,0.391965,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [10]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_lemmatized = tfidf_vectorizer.fit_transform(corpus_lemmatized)

In [11]:
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_lemmatized.toarray(), columns = feature_names)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yrs,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.742601,0.391988,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


**3. Натренируем gensim.models.Word2Vec модель на наших данных.**

●	Тренировать будем на токенизированных твитах `combine_df['tweet_token']`

●	Установим следующие параметры: `size=200`, `window=5`, `min_count=2`, `sg = 1`, `hs = 0`, `negative = 10`, `workers= 32`, `seed = 34`.

●	Используем функцию `train()` с параметром `total_examples` равным длине `combine_df['tweet_token']`, количество epochs установим 20.


In [12]:
sentences = [eval(str(item)) for item in data['tweet_token']]

In [13]:
sentences

[['when',
  'father',
  'is',
  'dysfunctional',
  'and',
  'is',
  'so',
  'selfish',
  'he',
  'drags',
  'his',
  'kids',
  'into',
  'his',
  'dysfunction',
  'run'],
 ['thanks',
  'for',
  'lyft',
  'credit',
  'can',
  'not',
  'use',
  'cause',
  'they',
  'do',
  'not',
  'offer',
  'wheelchair',
  'vans',
  'in',
  'pdx',
  'disapointed',
  'getthanked'],
 ['bihday', 'your', 'majesty'],
 ['model',
  'love',
  'yoyou',
  'take',
  'with',
  'yoyou',
  'all',
  'the',
  'time',
  'in',
  'yoyour'],
 ['factsguide', 'society', 'now', 'motivation'],
 ['huge',
  'fan',
  'fare',
  'and',
  'big',
  'talking',
  'before',
  'they',
  'leave',
  'chaos',
  'and',
  'pay',
  'disputes',
  'when',
  'they',
  'get',
  'there',
  'allshowandnogo'],
 ['camping', 'tomorrow', 'danny'],
 ['the',
  'next',
  'school',
  'year',
  'is',
  'the',
  'year',
  'for',
  'exams',
  'can',
  'not',
  'think',
  'about',
  'that',
  'school',
  'exams',
  'hate',
  'imagine',
  'actorslife',
  'revol

In [14]:
%%time
model = gensim.models.Word2Vec(sentences, size=200, window=5, min_count=1, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

Wall time: 9.66 s


In [15]:
%%time
model.train(sentences, total_examples=data['tweet_token'].shape[0], epochs=20)

Wall time: 15.1 s


(9763322, 11671440)

**4. Давайте немного потестируем нашу модель `Word2Vec` и посмотрим, как она работает. Мы зададим слово `positive = "dinner"`, и модель вытащит из корпуса наиболее похожие слова c помощью функции `most_similar`. То же самое попробуем со словом "trump".**

In [16]:
model.most_similar('dinner')

<ipython-input-16-7572d32f55f9>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('dinner')


[('thegafford', 0.5410997271537781),
 ('sleepnumber', 0.5360612869262695),
 ('eastvillage', 0.5279558300971985),
 ('cookout', 0.5274819135665894),
 ('roamingaround', 0.5231897830963135),
 ('teammatte', 0.5216646790504456),
 ('spaghetti', 0.5164714455604553),
 ('bolognese', 0.5095582008361816),
 ('firstvisit', 0.5084066987037659),
 ('fixings', 0.5078067183494568)]

In [17]:
model.most_similar('trump')

<ipython-input-17-d253bc5c5242>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('trump')


[('donald', 0.5941710472106934),
 ('suppoer', 0.5497506856918335),
 ('trumpleaks', 0.5457450151443481),
 ('dumptrump', 0.5442304611206055),
 ('unfit', 0.5436862111091614),
 ('cuck', 0.5422192811965942),
 ('trumptrain', 0.5394979119300842),
 ('fuhered', 0.5384372472763062),
 ('phony', 0.5314283967018127),
 ('melo', 0.5312877893447876)]

**5. Из приведенных выше примеров мы видим, что наша модель `word2vec` хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).**

Давайте проверим векторное представление любого слова из нашего корпуса, например `"food"`.

In [18]:
vector = model.wv['food']

In [19]:
vector

array([-0.71762556,  0.16148257, -1.064259  ,  0.55056024, -0.09022011,
       -0.67505467, -0.32063714, -0.30544695,  0.536679  ,  0.00941349,
       -0.5650636 , -0.4864554 , -0.44131747, -0.17802712, -0.05532002,
       -0.8189899 , -0.5788284 , -0.32957688,  0.6886659 , -0.00938168,
       -0.04149567, -0.00279074,  0.41424647, -0.06893849,  0.2213753 ,
       -0.982662  , -0.8104023 , -0.22912401, -0.29514   ,  0.03857312,
       -0.41548035,  0.10127994,  0.11560033,  0.5686998 , -0.18771099,
       -0.27121025, -0.09869439, -0.3388861 ,  0.8897826 , -0.08174036,
       -0.22300439,  0.32104874, -0.4186918 , -0.6632333 ,  0.08758989,
       -0.7633848 ,  1.0548328 ,  0.4415461 , -0.26736942, -0.01020163,
       -0.31487915, -0.15645953,  0.63238484, -0.36603427, -0.35070533,
        0.28294608,  0.3628884 ,  0.2544877 ,  0.34911734, -1.6235204 ,
       -0.29635975, -0.563272  ,  0.02282247,  0.6433265 , -0.49911523,
        0.00901926,  0.63486433,  0.7359646 ,  0.63296574, -0.50

**6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита.**

**Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.**

**Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size)) и поделить финальный вектор на количество слов в твите.**

**На выходе должен получиться wordvec_df.shape = (49159, 200).**

In [20]:
def mean_vec_tweet(sentences):
    vecs = []
    vec = 0
    for sentence in sentences:
        for word in sentence:
            vec += model.wv[word]
        vec = vec/len(sentence)
        vecs.append(vec)

    return np.array(vecs)

In [21]:
vecs = mean_vec_tweet(sentences)

<ipython-input-20-5ac75d75dbb4>:7: RuntimeWarning: divide by zero encountered in true_divide
  vec = vec/len(sentence)


In [22]:
vecs.shape

(49159, 200)